<a href="https://colab.research.google.com/github/alaoo/Hamoye-Regression/blob/main/logistic_Regression_Hamoye.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data_for_UCI_named.csv')

In [5]:
data.head

<bound method NDFrame.head of           tau1      tau2      tau3  ...        g4      stab     stabf
0     2.959060  3.079885  8.381025  ...  0.958034  0.055347  unstable
1     9.304097  4.902524  3.047541  ...  0.781760 -0.005957    stable
2     8.971707  8.848428  3.046479  ...  0.109853  0.003471  unstable
3     0.716415  7.669600  4.486641  ...  0.362718  0.028871  unstable
4     3.134112  7.608772  4.943759  ...  0.820923  0.049860  unstable
...        ...       ...       ...  ...       ...       ...       ...
9995  2.930406  9.487627  2.376523  ...  0.608385  0.023892  unstable
9996  3.392299  1.274827  2.954947  ...  0.366120 -0.025803    stable
9997  2.364034  2.842030  8.776391  ...  0.145984 -0.031810    stable
9998  9.631511  3.994398  2.757071  ...  0.818391  0.037789  unstable
9999  6.530527  6.781790  4.349695  ...  0.942631  0.045263  unstable

[10000 rows x 14 columns]>

In [6]:
df = data.drop('stab', axis=1)

In [7]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [8]:
df['stabf'] = df['stabf'].replace({'unstable': 0, 'stable': 1})

In [10]:
df['stabf'].head()

0    0
1    1
2    0
3    0
4    0
Name: stabf, dtype: int64

In [11]:
df['stabf'].shape

(10000,)

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import make_scorer, confusion_matrix
import xgboost as xgb

seed = 1


In [14]:
X = df.drop('stabf', axis=1)
y = df['stabf']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

y.value_counts()

0    6380
1    3620
Name: stabf, dtype: int64

In [16]:
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)

rf_clf = RandomForestClassifier(random_state=1)
rf_clf.fit(X_train, y_train)

y_pred_rf = rf_clf.predict(X_test)
print(rf_clf.score(X_test, y_test))
cm = confusion_matrix(y_test, y_pred_rf)
cm

0.9295


array([[1238,   50],
       [  91,  621]])

In [21]:
Ex_clf = ExtraTreesClassifier(random_state=1)
Ex_clf.fit(X_train, y_train)

y_pred_Ex = Ex_clf.predict(X_test)
print(Ex_clf.score(X_test, y_test))
cm_Ex = confusion_matrix(y_test, y_pred_Ex)
cm_Ex

0.9285


array([[1257,   31],
       [ 112,  600]])

In [23]:
xgb_clf = xgb.XGBClassifier(seed=seed)
xgb_clf.fit(X_train, y_train)

y_pred_xgb = xgb_clf.predict(X_test)
print(xgb_clf.score(X_test, y_test))
cm_xgb = confusion_matrix(y_test, y_pred_xgb)
cm_xgb

0.9195


array([[1236,   52],
       [ 109,  603]])

In [25]:
lgbm_clf = LGBMClassifier(seed=seed)
lgbm_clf.fit(X_train, y_train)

y_pred_lgbm = lgbm_clf.predict(X_test)
print(lgbm_clf.score(X_test, y_test))
cm_lgbm = confusion_matrix(y_test, y_pred_lgbm)
cm_lgbm

0.9375


array([[1240,   48],
       [  77,  635]])

In [26]:
n_estimators = [50, 100, 300, 500, 1000]

min_samples_split = [2, 3, 5, 7, 9]

min_samples_leaf = [1, 2, 4, 6, 8]

max_features = ['auto', 'sqrt', 'log2', None]

hyperparameter_grid = {'n_estimators': n_estimators,

'min_samples_leaf': min_samples_leaf,

'min_samples_split': min_samples_split,

'max_features': max_features}

ex2 = ExtraTreesClassifier(random_state=1)

random_cv = RandomizedSearchCV(ex2, param_distributions=hyperparameter_grid, scoring='accuracy', n_iter=10, cv=5, 
                                   verbose=1, n_jobs=-1, random_state=1)

random_cv.fit(X_train, y_train)

print(random_cv.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.2min finished


{'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 8, 'max_features': None}


In [29]:
print(random_cv.best_score_)
print(dict(zip(X_train.columns,random_cv.best_estimator_.feature_importances_)))
print(dict(zip(X_train.columns, Ex_clf.feature_importances_)))

0.9241249999999999
{'tau1': 0.13723974766109256, 'tau2': 0.14050750384993677, 'tau3': 0.13468028520386593, 'tau4': 0.1354167630909727, 'p1': 0.003683422151688322, 'p2': 0.005336864710946151, 'p3': 0.005429268421191957, 'p4': 0.004962486591192238, 'g1': 0.10256244080927947, 'g2': 0.10757764577478764, 'g3': 0.11306267999167334, 'g4': 0.10954089174337298}
{'tau1': 0.11739736493320078, 'tau2': 0.11844468079199041, 'tau3': 0.11316851246674982, 'tau4': 0.11546569217199552, 'p1': 0.039506754233827476, 'p2': 0.04037131556379323, 'p3': 0.040706278296424536, 'p4': 0.040578638540044426, 'g1': 0.08978290601483987, 'g2': 0.09367635844551439, 'g3': 0.09688268324346265, 'g4': 0.09401881529815702}
